# Previsão nota IMDB

O objetivo desta etapa do projeto é criar um modelo que preveja as notas da plataforma IMDB. Para testar, vamos inserir os dados de um novo filme e comparar com a nota real.

Este é um problema de regressão, pois o objetivo é uma variável continua. Vamos testar 3 modelos: Árvore de decisão, Floresta aleatória e Regressão Linear.


In [1]:
# Carregando as bibliotecas

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
import pickle

In [2]:
# Lendo os dados

df = pd.read_csv('https://raw.githubusercontent.com/pat-weber/lighthouse/main/lighthouse_project/data/desafio_indicium_imdb.csv')

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     999 non-null    int64  
 1   Series_Title   999 non-null    object 
 2   Released_Year  999 non-null    object 
 3   Certificate    898 non-null    object 
 4   Runtime        999 non-null    object 
 5   Genre          999 non-null    object 
 6   IMDB_Rating    999 non-null    float64
 7   Overview       999 non-null    object 
 8   Meta_score     842 non-null    float64
 9   Director       999 non-null    object 
 10  Star1          999 non-null    object 
 11  Star2          999 non-null    object 
 12  Star3          999 non-null    object 
 13  Star4          999 non-null    object 
 14  No_of_Votes    999 non-null    int64  
 15  Gross          830 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 125.0+ KB


In [3]:
# Padronizando as colunas
df.columns = df.columns.str.lower()
df['released_year'] = df['released_year'].replace('PG', 0)

# Transformando os dados dos anos em numéricos
df['released_year'] = df['released_year'].astype(int)
df['runtime'] = df['runtime'].str.replace('min', '').astype(int)
df['gross'] = df['gross'].str.replace(',', '').astype(float)

# Preenchendo valores ausentes
df['gross'] = df['gross'].fillna(df['gross'].median())
df['meta_score'] = df['meta_score'].fillna(df['meta_score'].median())

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   unnamed: 0     999 non-null    int64  
 1   series_title   999 non-null    object 
 2   released_year  999 non-null    int64  
 3   certificate    898 non-null    object 
 4   runtime        999 non-null    int64  
 5   genre          999 non-null    object 
 6   imdb_rating    999 non-null    float64
 7   overview       999 non-null    object 
 8   meta_score     999 non-null    float64
 9   director       999 non-null    object 
 10  star1          999 non-null    object 
 11  star2          999 non-null    object 
 12  star3          999 non-null    object 
 13  star4          999 non-null    object 
 14  no_of_votes    999 non-null    int64  
 15  gross          999 non-null    float64
dtypes: float64(3), int64(4), object(9)
memory usage: 125.0+ KB


Para usar dados que estão em string no data frame, vamos transformar em numéricos através de rótulos. Vamos olhar quantas entradas únicas existem nas colunas de diretores e atores.

In [4]:
# Conferindo quantos diretores e atores aparecem nas colunas

print('Diretores:', df['director'].nunique())
print('Atores principais:', df['star1'].nunique())
print('Atores 2:', df['star2'].nunique())
print('Atores 3:',df['star3'].nunique())
print('Atores 4:',df['star4'].nunique())

Diretores: 548
Atores principais: 659
Atores 2: 840
Atores 3: 890
Atores 4: 938


Vamos usar apenas as colunas de diretor e ator principal, pois as outras três colunas de atores tem poucas entradas que se repetem e dificilmente trará informação pertinente para os modelos. 

In [5]:
# Aplicando Label Encoder
le = LabelEncoder()

# Ajustando e transformando cada coluna de gênero
for col in ['certificate', 'genre', 'director', 'star1']:
    df[col] = le.fit_transform(df[col].astype(str))


As características que o modelo usará para a previsão são as colunas de classificação etária, ano de lançamento, tempo de duração, gênero, avaliações, número de votos e faturamento.

In [6]:
# Selecionando as características e objetivo

X = df[['certificate', 'released_year', 'runtime', 'genre', 'meta_score', 'no_of_votes', 'gross']]
y = df['imdb_rating']

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Vamos treinar os três modelos diferentes e analisar as métricas de performance.

### Modelo Floresta Aleatória

In [7]:
# Treinando o modelo de floresta aleatória
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Avaliar o modelo
y_pred_rf = rf_model.predict(X_test)
r2_rf = r2_score(y_test, y_pred_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)

print(f'R² Score (Floresta Aleatória): {r2_rf}')
print(f"MAE: {mae_rf}")

R² Score (Floresta Aleatória): 0.42332338065931463
MAE: 0.1512800000000002


### Modelo Árvore de Decisão

In [8]:
# Treinando o modelo de árvore de decisão
tree_model = DecisionTreeRegressor(random_state=42)
tree_model.fit(X_train, y_train)

# Avaliar o modelo
y_pred_tree = tree_model.predict(X_test)
r2_tree = r2_score(y_test, y_pred_tree)
mae_tree = mean_absolute_error(y_test, y_pred_tree)

print(f'R² Score (Árvore de Decisão): {r2_tree}')
print(f"MAE: {mae_tree}")

R² Score (Árvore de Decisão): 0.04180123088172594
MAE: 0.18700000000000003


### Modelo Regressão Linear

In [9]:
# Treinando o modelo de regressão linear

rl_model = LinearRegression()
rl_model.fit(X_train, y_train)

y_pred_rl = rl_model.predict(X_test)
r2_rl = r2_score(y_test, y_pred_rl)
mae_rl = mean_absolute_error(y_test, y_pred_rl)

print(f'R² Score (Regressão Linear): {r2_rl}')
print(f"MAE: {mae_rl}")

R² Score (Regressão Linear): 0.248463686315738
MAE: 0.18207241980919547


O modelo Árvore de decisão teve a pior perfomance com o valor R2 mais baixo e valor MAE mais alto. O modelo Floresta aleatória tem  o valor R2 mais alto, com 0.42, indicando que se ajusta melhor à variabilidade dos dados, o modelo Regressão linear tem o valor MAE ligeiramente mais baixo, indicando uma média menor de erros. Vamos testar estes dois modelos. 



In [10]:
# Lendo as informações do novo filme

new_movie = {
    'series_title': 'The Shawshank Redemption',
    'released_year': '1994',
    'certificate': 'A',
    'runtime': 142,
    'genre': 'Drama',
    'overview': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.',
    'meta_score': 80.0,
    'director': 'Frank Darabont',
    'star1': 'Tim Robbins',
    'star2': 'Morgan Freeman',
    'star3': 'Bob Gunton',
    'star4': 'William Sadler',
    'no_of_votes': 2343110,
    'gross': '28341469',
}

df_new_movie = pd.DataFrame([new_movie])

# Label Encoding para variáveis categóricas
label_encoder = LabelEncoder()
df_new_movie['certificate'] = label_encoder.fit_transform(df_new_movie['certificate'])
df_new_movie['genre'] = label_encoder.fit_transform(df_new_movie['genre'])

# Ajustar tipos de dados numéricos
df_new_movie['released_year'] = df_new_movie['released_year'].astype(int)
df_new_movie['meta_score'] = df_new_movie['meta_score'].astype(float)
df_new_movie['no_of_votes'] = df_new_movie['no_of_votes'].astype(int)
df_new_movie['gross'] = df_new_movie['gross'].astype(int)

# Extrair as características do novo filme
X_new_movie = df_new_movie[['certificate', 'released_year', 'runtime', 'genre', 'meta_score', 'no_of_votes', 'gross']]


In [11]:
# Floresta aleatória: previsão da nota do IMDb
imdb_rating_prediction_rf = rf_model.predict(X_new_movie)

print('Modelo Floresta Aleatória')
print(f"A previsão da nota do IMDb para o filme 'The Shawshank Redemption' é: {imdb_rating_prediction_rf}")

Modelo Floresta Aleatória
A previsão da nota do IMDb para o filme 'The Shawshank Redemption' é: [8.758]


In [12]:
# Regressão linear: previsão da nota do IMDb
imdb_rating_prediction_rl = rl_model.predict(X_new_movie)

print('Modelo Regressão Linear')
print(f"A previsão da nota do IMDb para o filme 'The Shawshank Redemption' é: {imdb_rating_prediction_rl}")

Modelo Regressão Linear
A previsão da nota do IMDb para o filme 'The Shawshank Redemption' é: [9.15818103]


A nota real do filme no IMDB é 9.3. Nesse caso, o modelo de Floresta Aleatória apresentou um erro maior do que o modelo de Regressão Linear, que obteve um resultado mais próximo da nota real.

Considerando que estamos lidando com notas de 0 a 10 e que desejamos que todos os erros tenham o mesmo peso, a métrica MAE é a mais adequada. Portanto, recomendamos a utilização do modelo de Regressão Linear para esta aplicação.

In [13]:
# Salvando o modelo

import pickle
with open("rl_model.pkl", 'wb') as f:
    pickle.dump(rl_model, f)